# **Clustering Online Retail Data**

The code below is taken from Manish Kumar's submission on [kaggle.com](https://www.kaggle.com/hellbuoy/online-retail-k-means-hierarchical-clustering/notebook).

You are encouraged to go to the link above and check the full code. In this lab, you will do the necessary steps to explore the data and prepare it for sklearn algorithms.

**About the data set**

Online retail is a transnational data set which contains all the transactions occurring between 01/12/2010 and 09/12/2011 for a UK-based and registered non-store online retail. The company mainly sells unique all-occasion gifts. Many customers of the company are wholesalers.

Using K-Means Clustering, you can segement the Customers based on RFM so that the company can target its customers efficiently.

**Import libraries**

In [2]:
# import required libraries for dataframe and visualization

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # for data visualization
import seaborn as sns # for statistical data visualization
import datetime as dt

# import required sklearn libraries for KMeans Clustering
import sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans

# Acquire data

In [3]:
# Read in the data using panda's read_csv method
retail = pd.read_csv('UnsupervisedLearning/OnlineRetail/OnlineRetail.csv', sep=",", encoding="ISO-8859-1", header=0)

#TODO: Write code to inspect the first five rows of the data frame
retail.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,01-12-2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,01-12-2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,01-12-2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,01-12-2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,01-12-2010 8:26,3.39,17850.0,United Kingdom


# Inspect data

In [4]:
#TODO: Write code to inspect the shape of the data frame
retail.shape

(2500, 8)

In [5]:
#TODO: Write code to display information about the data frame
retail.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2500 entries, 0 to 2499
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   InvoiceNo    2500 non-null   object 
 1   StockCode    2500 non-null   object 
 2   Description  2490 non-null   object 
 3   Quantity     2500 non-null   int64  
 4   InvoiceDate  2500 non-null   object 
 5   UnitPrice    2500 non-null   float64
 6   CustomerID   1919 non-null   float64
 7   Country      2500 non-null   object 
dtypes: float64(2), int64(1), object(5)
memory usage: 156.4+ KB


In [6]:
#TODO: Write code to display statistics about the data frame
retail.describe()

,Quantity,UnitPrice,CustomerID
count,2500.000000,2500.000000,1919.000000
mean,10.065200,3.701044,15637.112559
std,29.139317,12.328907,1848.418705
min,-24.000000,0.000000,12431.000000
25%,1.000000,1.250000,14307.000000
50%,3.000000,2.510000,15605.000000
75%,10.000000,4.210000,17841.000000
max,600.000000,569.770000,18229.000000


# Clean data

**Correcting**

In [7]:
#TODO: Write code to drop rows having missing values
retail = retail.dropna()

In [7]:
# Changing the datatype of Customer Id to a string as per Business understanding

retail['CustomerID'] = retail['CustomerID'].astype(str)

**Creating**

We are going to analysis the Customers based on 3 factors:
* R (Recency): Number of days since last purchase
* F (Frequency): Number of transactions
* M (Monetary): Total amount of transactions (revenue contributed)

In [8]:
# Create a new attribute : Monetary based on Quantity and UnitPrice

retail['Amount'] = retail['Quantity']*retail['UnitPrice']
rfm_m = retail.groupby('CustomerID')['Amount'].sum()
rfm_m = rfm_m.reset_index()

#TODO: Write code to inspect the first five rows of rfm_m
rfm_m.head()

,CustomerID,Amount
0,12431.0,358.25
1,12433.0,1919.14
2,12472.0,-122.30
3,12583.0,855.86
4,12662.0,261.48


In [10]:
# Create a new attribute : Frequency based on invoice count per customer ID

rfm_f = retail.groupby('CustomerID')['InvoiceNo'].count()
rfm_f = rfm_f.reset_index()
rfm_f.columns = ['CustomerID', 'Frequency']

#TODO: Write code to inspect the first five rows of rfm_f
rfm_f.head()

,CustomerID,Frequency
0,12431.0,14
1,12433.0,73
2,12472.0,14
3,12583.0,20
4,12662.0,15


In [11]:
# Merge the two data frames

rfm = pd.merge(rfm_m, rfm_f, on='CustomerID', how='inner')

#TODO: Write code to inspect the first five rows of rfm
rfm.head()

,CustomerID,Amount,Frequency
0,12431.0,358.25,14
1,12433.0,1919.14,73
2,12472.0,-122.30,14
3,12583.0,855.86,20
4,12662.0,261.48,15


In [12]:
# Create a new attribute : Recency
# -----------------------------------

# Convert to datetime to proper datatype
retail['InvoiceDate'] = pd.to_datetime(retail['InvoiceDate'],format='%d-%m-%Y %H:%M')

# Compute the maximum date to know the last transaction date
max_date = max(retail['InvoiceDate'])

# Compute the difference between max date and transaction date
retail['Diff'] = max_date - retail['InvoiceDate']

# Compute last transaction date to get the recency of customers
rfm_p = retail.groupby('CustomerID')['Diff'].min()
rfm_p = rfm_p.reset_index()

# Extract number of days only
rfm_p['Diff'] = rfm_p['Diff'].dt.days

# Merge tha dataframes to get the final RFM dataframe
rfm = pd.merge(rfm, rfm_p, on='CustomerID', how='inner')
rfm.columns = ['CustomerID', 'Amount', 'Frequency', 'Recency']

#TODO: Write code to inspect the first five rows of the data frame
rfm.head()

,CustomerID,Amount,Frequency,Recency
0,12431.0,358.25,14,0
1,12433.0,1919.14,73,0
2,12472.0,-122.30,14,0
3,12583.0,855.86,20,0
4,12662.0,261.48,15,0


**Remove outliers**

In [12]:
# Outlier Analysis of Amount Frequency and Recency

attributes = ['Amount','Frequency','Recency']
plt.rcParams['figure.figsize'] = [10,8]
sns.boxplot(data = rfm[attributes], orient="v", palette="Set2" ,whis=1.5,saturation=1, width=0.7)
plt.title("Outliers Variable Distribution", fontsize = 14, fontweight = 'bold')
plt.ylabel("Range", fontweight = 'bold')
plt.xlabel("Attributes", fontweight = 'bold')

In [14]:
# Remove outliers for Amount
Q1 = rfm.Amount.quantile(0.05)
Q3 = rfm.Amount.quantile(0.95)
IQR = Q3 - Q1
rfm = rfm[(rfm.Amount >= Q1 - 1.5*IQR) & (rfm.Amount <= Q3 + 1.5*IQR)]

#TODO: Write code to remove outliers for Recency
Q1 = rfm.Recency.quantile(0.05)
Q3 = rfm.Recency.quantile(0.95)
IQR = Q3 - Q1
rfm = rfm[(rfm.Recency >= Q1 - 1.5*IQR) & (rfm.Recency <= Q3 + 1.5*IQR)]

# Remove outliers for Frequency
Q1 = rfm.Frequency.quantile(0.05)
Q3 = rfm.Frequency.quantile(0.95)
IQR = Q3 - Q1
rfm = rfm[(rfm.Frequency >= Q1 - 1.5*IQR) & (rfm.Frequency <= Q3 + 1.5*IQR)]

**Rescale attributes**

In [15]:
# Rescaling the attributes using a MinMaxScaler

rfm_df = rfm[['Amount', 'Frequency', 'Recency']]

# Instantiate
scaler = MinMaxScaler()

# fit_transform
rfm_df_scaled = scaler.fit_transform(rfm_df)

rfm_df_scaled = pd.DataFrame(rfm_df_scaled)
rfm_df_scaled.columns = ['Amount', 'Frequency', 'Recency']
rfm_df_scaled.head()

,Amount,Frequency,Recency
0,0.191012,0.154762,0.0
1,0.787633,0.857143,0.0
2,0.007331,0.154762,0.0
3,0.381214,0.226190,0.0
4,0.154024,0.166667,0.0


# Earn Your Wings

Implement a K-Means Clustering algorithm on the cleaned data set. Use the elbow method to find the right value of k to use.
Add comments in your code to explain each step that you take in your implementation.